In [1]:
import numpy as np
import pybullet as p
import time
from robot_descriptions.loaders.pybullet import load_robot_description

pybullet build time: Nov 29 2024 16:26:49


In [ ]:
p.connect(p.GUI_SERVER)
p.setTimeStep(0.001)
p.setGravity(0, 0, -9.81)

In [7]:
#robot_id = load_robot_description('ur5_description', useFixedBase=True)
joint_indices = [i for i in range(p.getNumJoints(robot_id)) if p.getJointInfo(robot_id, i)[2] == p.JOINT_REVOLUTE]
for i in range(p.getNumJoints(robot_id)):
  print(p.getJointInfo(robot_id, i))

eff_idx = 7
print("End effector joint: ", eff_idx)
print("Revolute joints: ", joint_indices)

(0, b'world_joint', 4, -1, -1, 0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, b'base_link', (0.0, 0.0, 0.0), (0.0, 0.0, 0.0), (0.0, 0.0, 0.0, 1.0), -1)
(1, b'shoulder_pan_joint', 0, 7, 6, 1, 0.0, 0.0, -6.28318530718, 6.28318530718, 150.0, 3.15, b'shoulder_link', (0.0, 0.0, 1.0), (0.0, 0.0, 0.089159), (0.0, 0.0, 0.0, 1.0), 0)
(2, b'shoulder_lift_joint', 0, 8, 7, 1, 0.0, 0.0, -6.28318530718, 6.28318530718, 150.0, 3.15, b'upper_arm_link', (0.0, 1.0, 0.0), (0.0, 0.13585, 0.0), (0.0, -0.7071067811848163, 0.0, 0.7071067811882787), 1)
(3, b'elbow_joint', 0, 9, 8, 1, 0.0, 0.0, -3.14159265359, 3.14159265359, 150.0, 3.15, b'forearm_link', (0.0, 1.0, 0.0), (0.0, -0.1197, 0.14499999999999996), (0.0, 0.0, 0.0, 1.0), 2)
(4, b'wrist_1_joint', 0, 10, 9, 1, 0.0, 0.0, -6.28318530718, 6.28318530718, 28.0, 3.2, b'wrist_1_link', (0.0, 1.0, 0.0), (0.0, 0.0, 0.14225), (0.0, -0.7071067811848163, 0.0, 0.7071067811882787), 3)
(5, b'wrist_2_joint', 0, 11, 10, 1, 0.0, 0.0, -6.28318530718, 6.28318530718, 28.0, 3.2, b'wrist_2_l

In [2]:
from robot_descriptions import ur5_description
urdf_file_path_ur5 = ur5_description.URDF_PATH
print(urdf_file_path_ur5)

/Users/theo/.cache/robot_descriptions/example-robot-data/robots/ur_description/urdf/ur5_robot.urdf


# Disable default motor control

In [8]:
for i in joint_indices:
    p.setJointMotorControl2(
        bodyUniqueId=robot_id,
        jointIndex=i,
        controlMode=p.VELOCITY_CONTROL,
        force=0  # Disable motor forces
    )

# Feedback and Stabilization

In [10]:
def pd_control(robot_id, q_desired, v_desired, Kp, Kd):
  joint_states = p.getJointStates(robot_id, joint_indices)
  q_measured = np.array([state[0] for state in joint_states])
  v_measured = np.array([state[1] for state in joint_states])
  position_error = q_desired - q_measured
  velocity_error = v_desired - v_measured
  
  tau = Kp * position_error + Kd * velocity_error
  # add gravity compensation
  tau += np.array(p.calculateInverseDynamics(robot_id, q_measured.tolist(), [0] * len(joint_indices), [0] * len(joint_indices)))
  return tau

Kp = 100 * np.ones_like(joint_indices)
Kd = 5 * np.ones_like(joint_indices)

def control_to_desired_configuration(robot_id, q_desired, v_desired=None, max_steps=5000):
  if v_desired is None:
    v_desired = np.zeros_like(q_desired)
  for step in range(max_steps):  # Simulation steps
    torques = pd_control(robot_id, q_desired, v_desired, Kp, Kd).tolist()
    p.setJointMotorControlArray(
      bodyUniqueId=robot_id,
      jointIndices=joint_indices,
      controlMode=p.TORQUE_CONTROL,
      forces=torques
    )
    p.stepSimulation()
    

# Test PD controller

In [13]:
desired_positions = np.array([0.5, -1.0, 0.0, -1.5, 0.0, 1.0])
desired_velocities = np.zeros_like(desired_positions)

control_to_desired_configuration(robot_id, desired_positions, desired_velocities)

# Moving end-effector along a horizontal line

In [6]:
# Define the start point and direction of the line
start_point = np.array([0.8, 0.3, 0.3])  # [x, y, z] in 3D space
direction_vector = np.array([0.0, -0.6, 0])  # Direction along X-axis (horizontal)
target_positions = start_point + np.linspace(0, 1, num=100)[:, np.newaxis] * direction_vector[np.newaxis, :]
print(target_positions[0], target_positions[1])
print(p.getJointInfo(robot, eff_idx))
# set robot at starting point
time.sleep(2)
for target in target_positions:
  joint_angles = p.calculateInverseKinematics(
    robot,
    eff_idx,
    target
  )
  control_to_desired_configuration(robot, joint_angles)
  time.sleep(1 / 240.0)

[0.8 0.3 0.3] [0.8        0.29393939 0.3       ]


NameError: name 'eff_idx' is not defined

## Adding pendulum to end effector

In [3]:
import os



def addpendulum_urdf(urdf_file_path, save_to_path):
    # Read the URDF file
    with open(urdf_file_path, 'r') as file:
        urdf_data = file.readlines()

    # Remove trailing blank lines and the </robot> tag
    while urdf_data and urdf_data[-1].strip() == "":  # Remove blank lines
        urdf_data.pop()
    # Remove the last line (`</robot>`)
    if urdf_data[-1].strip() == "</robot>":
        urdf_data.pop()


    # The pendulum components to add
    pendulum_urdf = """  <!-- Pendulum Link (Thin cylinder attached to link_6) -->
  <link name="pendulum_link">
    <inertial>
      <origin xyz="0 0 -0.125" rpy="0 0 0" />
      <mass value="0.005" />
      <inertia ixx="0.0001" ixy="0" ixz="0" iyy="0.0000" iyz="0" izz="0.0001" /> <!-- ixx =izz = m * l **2 /3, rest is 0 (source wiki)  -->
    </inertial>
    <visual>
      <origin xyz="0 0 -0.125" rpy="0 0 0" />
      <geometry>
      <cylinder length="0.25" radius="0.0025"/> <!-- Thin cylinder, length 0.25m, radius 2.5mm -->
      </geometry>
      <material name="">
        <color rgba="0.792156862745098 0.819607843137255 0.933333333333333 1" />
      </material>
    </visual>
    <collision>
      <origin xyz="0 0 -0.125" rpy="0 0 0" /> 
      <geometry>
      <cylinder length="0.25" radius="0.0025"/> <!-- Thin cylinder, length 0.25m, radius 2.5mm -->
      </geometry>
    </collision>
  </link>

  <!-- Pendulum Mass at the end of the pendulum_link -->
  <link name="pendulum_mass">
    <inertial>
      <origin xyz="0 0 -0.03" rpy="0 0 0" />
      <mass value="0.001" /> <!-- Mass m at the end of 100g -->
      <inertia ixx="0.000036" ixy="0" ixz="0" iyy="0.000036" iyz="0" izz="0.000036" /> <!-- ixx = iyy=izz = 2 *m * r **2 /5 , rest is 0 (source wiki)   -->
    </inertial>
    <visual>
      <origin xyz="0 0 -0.03" rpy="0 0 0" />
      <geometry>
        <sphere radius ="0.03" /> <!-- Small sphere representing the mass, radius of 3cm -->
      </geometry>
      <material name="">
        <color rgba="0.792156862745098 0.819607843137255 0.933333333333333 1" />
      </material>
    </visual>
    <collision>
      <origin xyz="0 0 -0.03" rpy="0 0 0" />
      <geometry>
        <sphere radius="0.03" />
      </geometry>
    </collision>
  </link>

  <!-- Pendulum Joint (connects pendulum_link to link_6) -->
  <joint name="pendulum_joint" type="fixed">
    <parent link="ee_link" /> <!-- Attach to link_6 -->
    <child link="pendulum_link" />
    <origin xyz="0 0 0" rpy="0 3.1416 0" /> <!-- Positioning pendulum at link_6's end -->
    <axis xyz="0 1 0" /> <!-- Swinging along Y-axis -->
    <limit lower="-0.4" upper="0.4" effort="0" velocity="1" /> <!-- Passive joint with zero effort -->
    <dynamics damping="0.1" friction="0.01" />
  </joint>

  <!-- Pendulum Mass Joint (fixes mass to the end of the pendulum) -->
  <joint name="pendulum_mass_joint" type="fixed">
    <parent link="pendulum_link" />
    <child link="pendulum_mass" />
    <origin xyz="0 0 -0.25" rpy="0 0 0" /> <!-- Attach mass at the tip of pendulum -->
  </joint>
  """
    # Append the pendulum components to the URDF data
    urdf_data.append(pendulum_urdf)

    # Add the </robot> tag back
    urdf_data.append("</robot>\n")

    # Write the modified URDF back to the file
    with open(save_to_path, 'w') as file:
        file.writelines(urdf_data)

    print("URDF file has been updated with pendulum components.")
    print(f"New URDF saved to {save_to_path}")




In [4]:
pendulum_arm_path = os.path.dirname(urdf_file_path_ur5) + '/ur5_pendulum.urdf'

addpendulum_urdf(urdf_file_path_ur5, save_to_path= pendulum_arm_path)

URDF file has been updated with pendulum components.
New URDF saved to /Users/theo/.cache/robot_descriptions/example-robot-data/robots/ur_description/urdf/ur5_pendulum.urdf


In [5]:
p.connect(p.GUI_SERVER)
p.setTimeStep(0.001)
p.setGravity(0, 0, -9.81)

robot_id = p.loadURDF(pendulum_arm_path)

Version = 4.1 Metal - 76.3
Vendor = Apple
Renderer = Apple M1
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
b3Printf: PosixSharedMemory::releaseSharedMemory removed shared memory
b3Printf: PosixSharedMemory::releaseSharedMemory detached shared memory

b3Printf: PosixSharedMemory::releaseSharedMemory removed shared memory
b3Printf: PosixSharedMemory::releaseSharedMemory detached shared memory

MotionThreadFunc thread started
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: urdfdom: no axis element for Joint, defaulting to (1,0,0) axis ee_fixed_joint

